In [1]:
import sys
import os
import inspect

import re
import json
import statistics
import argparse
import itertools
from pathlib import Path
from dataclasses import dataclass

# script_dir: str = os.path.dirname(os.path.abspath(__file__))
# os.chdir(script_dir)

# Get the directory of the current notebook and cd into it
# notebook_path = os.path.basename(sys.argv[0])
# notebook_dir = os.path.dirname(os.path.abspath(notebook_path))

# notebook_path = os.path.abspath(inspect.getfile(inspect.currentframe()))
# notebook_dir = os.path.dirname(notebook_path)

# os.chdir(notebook_dir)
# print(f"{sys.argv=}")

os.chdir('./scripts')

print(f"{os.getcwd()=}")

from ldj import ldj
from utils import *

import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
from matplotlib.patches import PathPatch
from matplotlib.path import get_path_collection_extents
import seaborn as sns

from rich import print, pretty
from tabulate import tabulate
from typing import  Iterable
import pretty_errors
from catppuccin import PALETTE
from IPython.display import display, HTML

pretty.install()

EXPERIMENT_DIR = Path("../experiments/collaborative-complex")
assert EXPERIMENT_DIR.is_dir() and EXPERIMENT_DIR.exists()

flavor = PALETTE.latte.colors

data = dict()

@dataclass
class Results:
    with_tracking: dict
    without_tracking: dict

results = Results(dict(), dict())

with open(EXPERIMENT_DIR / "tracking-true.json") as f:
    results.with_tracking = json.load(f)

with open(EXPERIMENT_DIR / "tracking-false.json") as f:
    results.without_tracking = json.load(f)


os.getcwd()='/home/kpbaks/university/semesters/10/master-thesis/gbp-rs/scripts'


sys.executable = '/home/kpbaks/.nix-profile/bin/python'

sys.version = '3.11.9 (main, Apr  2 2024, 08:25:04) [GCC 13.3.0]'

In [2]:
@dataclass(frozen=True)
class Statistics:
    mean: float
    median: float
    stdev: float
    min: float
    max: float


    def display(self) -> None:
        data = [
            ["Mean", self.mean],
            ["Median", self.median],
            ["Standard Deviation", self.stdev],
            ["Min", self.min],
            ["Max", self.max]
        ]
        html_table = tabulate(data, headers=["Statistic", "Value"], tablefmt="html")
        centered_html_table = f"""
        <div style="display: flex; justify-content: center;">
            {html_table}
        </div>
        """
        # display(HTML(html_table))
        display(HTML(centered_html_table))
        # print(tabulate(data, headers=["Statistic", "Value"], tablefmt="html"))


def compute_stats(data: list[float]) -> Statistics:
    return Statistics(
        mean=np.mean(data),
        median=np.median(data),
        stdev=np.std(data),
        min=np.min(data),
        max=np.max(data),
    )


In [3]:
@dataclass(frozen=True)
class PerpendicularPositionErrorResult:
    errors: list[float]
    rmses: list[float]


def perpendicular_position_error(exported_data: dict) -> PerpendicularPositionErrorResult:
    errors: list[float] = []
    rmses: list[float] = []

    for robot_id, robot_data in exported_data['robots'].items():
        color: str = robot_data['color']
        positions = np.array([p for p in robot_data['positions']])
        mission = robot_data['mission']
        waypoints = []
        for route in mission['routes']:
            waypoints.append(route['waypoints'][0])
            for wp in route['waypoints'][1:]:
                waypoints.append(wp)

        waypoints = np.array(waypoints)
        waypoints = np.squeeze(waypoints)

        lines: list[LinePoints] = [LinePoints(start=start, end=end) for start, end in sliding_window(waypoints, 2)]
        closest_projections = [closest_projection_onto_line_segments(p, lines) for p in positions]

        error: float = np.sum(np.linalg.norm(positions - closest_projections, axis=1))
        rmse: float = np.sqrt(error / len(positions))

        errors.append(error)
        rmses.append(rmse)

    return PerpendicularPositionErrorResult(errors=errors, rmses=rmses)


In [4]:
@dataclass(frozen=True)
class CollisionsResult:
    interrobot: int
    environment: int

def collisions(exported_data: dict) -> CollisionsResult:
    interrobot: int = len(exported_data['collisions']['robots'])
    environment: int = len(exported_data['collisions']['environment'])
    return CollisionsResult(interrobot=interrobot, environment=environment)


# With Tracking

## Makespan

In [5]:
print(f"makespan = {results.with_tracking['makespan']:.2f} seconds")


makespan = 158.03 seconds

## Perpendicular Position Error

In [6]:
pperror = perpendicular_position_error(results.with_tracking)
compute_stats(pperror.errors).display()


Statistic,Value
Mean,114.416
Median,106.453
Standard Deviation,49.7417
Min,0.007467
Max,314.922


## Collisions

In [7]:
collisions(results.with_tracking)


CollisionsResult(interrobot=0, environment=0)

# Without Tracking

In [8]:
print(f"makespan = {results.without_tracking['makespan']:.2f} seconds")


makespan = 158.13 seconds

## Perpendicular Position Error

In [9]:
pperror = perpendicular_position_error(results.without_tracking)
compute_stats(pperror.errors).display()


Statistic,Value
Mean,162.018
Median,156.232
Standard Deviation,64.8727
Min,9.64423e-13
Max,393.413


## Collisions

In [10]:
collisions(results.without_tracking)


CollisionsResult(interrobot=0, environment=0)